Check if the TPU is available (only if TPU was selected as hardware accelerator).

In [1]:
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

**Installation:**
Clone our repository and to get the python scripts and the datasets for probing.

In [ ]:
!git clone https://github.com/SwiftPredator/How-Does-Bert-Answer-QA-DLP2021/
!mv "/content/How-Does-Bert-Answer-QA-DLP2021/src/probing-tasks/data" "/content/"
!mv "/content/How-Does-Bert-Answer-QA-DLP2021/src/probing-tasks/replicate" "/content/"

%cd /content/replicate
!pip install -r requirements.txt
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

Restart the runtime and import the libraries. If the runtime is restarted at any point, it suffices to run from here on.

In [ ]:
%cd /content/replicate

from edge_probing_utils import (
    JiantDatasetSingleSpan,
    JiantDatasetTwoSpan
    )

import edge_probing as ep
import edge_probing_tpu as ep_tpu
import torch
import torch.nn as nn
import torch.utils.data as data
from transformers import AutoModel, AutoTokenizer

**Setup:**
Select the tasks and models to run and pick a dataset size. Since the time for a probing tasks depends on the patience, patience_lr and max_evals parameters in the ProbeConfig, it is adviced to always select the big dataset.

In [ ]:

tasks = [
    #"ner", 
    #"semeval",
    "coref",
    "sup-squad",
    "ques",
    #"sup-babi",
    "adversarialqa",
    ]

task_types = {
    "ner": "single_span", 
    "semeval": "single_span",
    "coref": "two_span",
    "sup-squad": "two_span",
    "ques": "single_span",
    "sup-babi": "two_span",
    "adversarialqa": "two_span",
    }

models = [
    "roberta-base",
    #"bert-base-uncased",
    #"csarron/bert-base-uncased-squad-v1"
    #"/content/drive/MyDrive/BERT_fineTunned_on_AdversarialQA_trained"
    ]

task_labels_to_ids = {
    "ner": {'ORDINAL': 0, 'DATE': 1, 'PERSON': 2, 'LOC': 3, 'GPE': 4, 'QUANTITY': 5, 'ORG': 6, 'WORK_OF_ART': 7, 'CARDINAL': 8, 'TIME': 9, 'MONEY': 10, 'LANGUAGE': 11, 'NORP': 12, 'PERCENT': 13, 'EVENT': 14, 'LAW': 15, 'FAC': 16, 'PRODUCT': 17},
    "coref": {"0": 0, "1": 1},
    "semeval": {'Component-Whole(e2,e1)': 0, 'Other': 1, 'Instrument-Agency(e2,e1)': 2, 'Member-Collection(e1,e2)': 3, 'Entity-Destination(e1,e2)': 4, 'Content-Container(e1,e2)': 5, 'Message-Topic(e1,e2)': 6, 'Cause-Effect(e2,e1)': 7, 'Product-Producer(e2,e1)': 8, 'Member-Collection(e2,e1)': 9, 'Entity-Origin(e1,e2)': 10, 'Cause-Effect(e1,e2)': 11, 'Component-Whole(e1,e2)': 12, 'Message-Topic(e2,e1)': 13, 'Product-Producer(e1,e2)': 14, 'Entity-Origin(e2,e1)': 15, 'Content-Container(e2,e1)': 16, 'Instrument-Agency(e1,e2)': 17, 'Entity-Destination(e2,e1)': 18},
    "sup-squad": {"0": 0, "1": 1},
    "ques": {'LOC:other': 0, 'DESC:desc': 1, 'DESC:def': 2, 'DESC:manner': 3, 'ENTY:sport': 4, 'ENTY:termeq': 5, 'HUM:ind': 6, 'NUM:count': 7, 'DESC:reason': 8, 'LOC:country': 9, 'HUM:desc': 10, 'ENTY:animal': 11, 'ENTY:other': 12, 'LOC:city': 13, 'ENTY:cremat': 14, 'NUM:perc': 15, 'NUM:money': 16, 'NUM:date': 17, 'ENTY:dismed': 18, 'LOC:state': 19, 'NUM:speed': 20, 'HUM:gr': 21, 'NUM:dist': 22, 'ENTY:food': 23, 'ABBR:abb': 24, 'ENTY:product': 25, 'HUM:title': 26, 'NUM:weight': 27, 'ABBR:exp': 28, 'ENTY:veh': 29, 'NUM:period': 30, 'ENTY:religion': 31, 'ENTY:letter': 32, 'ENTY:color': 33, 'ENTY:body': 34, 'ENTY:event': 35, 'ENTY:substance': 36, 'ENTY:instru': 37, 'ENTY:plant': 38, 'ENTY:symbol': 39, 'NUM:other': 40, 'LOC:mount': 41, 'NUM:temp': 42, 'ENTY:techmeth': 43, 'NUM:code': 44, 'ENTY:word': 45, 'ENTY:lang': 46, 'NUM:volsize': 47, 'NUM:ord': 48, 'ENTY:currency': 49},
    "sup-babi": {"0": 0, "1": 1},
    "adversarialqa": {"0": 0, "1": 1},
    }

# Pick a dataset size: small, medium, big
size = "big"

**Run:**
Run the selected probing tasks. Select the layers to probe and the device. If another device than the TPU is used make sure to comment out the first two imports (torch_xla and torch_xla.core.xla_model) and set tpu to False. Furthermore the import edge_probing_tpu two cells above needs to be commented out.

The results of the run are saved to /content/drive/MyDrive/replicated/probing-results/{size}.json. This file is updated everytime a task was probed on all selected layers. If for some reason the notebook fails to probe all layers, the intermediate results of the last task can be found at the specified results_path (by default /content/drive/MyDrive/replicated/intermediate-results/results.json).

The probing for the report was done with different big datasets (the size was adjusted afterwards and they were shuffled). To reproduce those results set original to True. Note that the dataset size has to be set to big for this.

In [ ]:
import torch_xla
import torch_xla.core.xla_model as xm
import os
import json
import matplotlib.pyplot as plt
from google.colab import drive

drive.mount('/content/drive')

loss_function = nn.BCELoss()
batch_size = 32
num_layers = range(1,13,2)
num_workers = 0

#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = xm.xla_device()
tpu = True
#device = torch.device("cpu")

original = False

# Disable warnings.
os.environ["TOKENIZERS_PARALLELISM"] = "false"

os.makedirs(f"/content/drive/MyDrive/replicated/probing-results/", exist_ok=True)
os.makedirs(f"/content/drive/MyDrive/replicated/intermediate-results/", exist_ok=True)

for model in models:
    tokenizer = AutoTokenizer.from_pretrained(model)
    for task in tasks:
        if os.path.isfile(f'/content/drive/MyDrive/replicated/probing-results/{size}.json'):
            with open(f'/content/drive/MyDrive/replicated/probing-results/{size}.json', 'r') as f:
                results = json.load(f)
        else:
            results = {}
        task_results = results.setdefault(task, {})
        labels_to_ids = task_labels_to_ids[task]
        if original:
            train_data = ep.tokenize_jiant_dataset(
                tokenizer,
                *(ep.read_jiant_dataset(f"/content/data/{task}/{size}/original/train.jsonl")),
                labels_to_ids,
                )
            val_data = ep.tokenize_jiant_dataset(
                tokenizer,
                *(ep.read_jiant_dataset(f"/content/data/{task}/{size}/original/val.jsonl")),
                labels_to_ids,
                )
            test_data = ep.tokenize_jiant_dataset(
                tokenizer,
                *(ep.read_jiant_dataset(f"/content/data/{task}/{size}/original/val.jsonl")),
                labels_to_ids,
                )
        else:
            train_data = ep.tokenize_jiant_dataset(
                tokenizer,
                *(ep.read_jiant_dataset(f"/content/data/{task}/{size}/train.jsonl")),
                labels_to_ids,
                )
            val_data = ep.tokenize_jiant_dataset(
                tokenizer,
                *(ep.read_jiant_dataset(f"/content/data/{task}/{size}/val.jsonl")),
                labels_to_ids,
                )
            test_data = ep.tokenize_jiant_dataset(
                tokenizer,
                *(ep.read_jiant_dataset(f"/content/data/{task}/{size}/test.jsonl")),
                labels_to_ids,
                )
        if task_types[task] == "single_span":
            train_data = JiantDatasetSingleSpan(train_data)
            val_data = JiantDatasetSingleSpan(val_data)
            test_data = JiantDatasetSingleSpan(test_data)
        elif task_types[task] == "two_span":
            train_data = JiantDatasetTwoSpan(train_data)
            val_data = JiantDatasetTwoSpan(val_data)
            test_data = JiantDatasetTwoSpan(test_data)
        train_loader = data.DataLoader(train_data, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=num_workers)
        val_loader = data.DataLoader(val_data, batch_size=batch_size, pin_memory=True, num_workers=num_workers)
        test_loader = data.DataLoader(test_data, batch_size=batch_size, pin_memory=True, num_workers=num_workers)
        if tpu:
            results[task][model] = ep_tpu.probing(ep.ProbeConfig(
                train_loader,
                val_loader,
                test_loader,
                model,
                num_layers,
                loss_function,
                labels_to_ids,
                task_types[task],
                lr=0.0001,
                patience=5,
                eval_interval=1000,
                dev=device,
                results_path="/content/drive/MyDrive/replicated/intermediate-results/",
                ))
        else:
            results[task][model] = ep.probing(ep.ProbeConfig(
                train_loader,
                val_loader,
                test_loader,
                model,
                num_layers,
                loss_function,
                labels_to_ids,
                task_types[task],
                lr=0.0001,
                patience=5,
                eval_interval=1000,
                dev=device,
                results_path="/content/drive/MyDrive/replicated/intermediate-results/",
                ))
        with open(f'/content/drive/MyDrive/replicated/probing-results/{size}.json', 'w') as f:
            json.dump(results, f)

**Visualize the results:**

In [ ]:
model_to_label = {
    "roberta-base": "RoBERTa",
    "bert-base-uncased": "no fine-tuning", 
    "csarron/bert-base-uncased-squad-v1": "SQuAD",
    "/content/drive/MyDrive/BERT_fineTunned_on_AdversarialQA_trained": "adversarialQA",
    }
model_to_linestyle = {
    "bert-base-uncased": ":g", 
    "csarron/bert-base-uncased-squad-v1": "-y",
    "/content/drive/MyDrive/BERT_fineTunned_on_AdversarialQA_trained": "--b",
    "roberta-base": "-.r",
    }

task_to_title = {
    "coref": "COREF",
    "ques": "Question Type",
    "sup-squad": "Supporting Facts SQuAD",
    "sup-babi": "Supporting Facts bAbI",
    "ner": "Named Entity Labeling",
    "semeval": "Relation Classification",
    "adversarialqa": "Adversarial QA"
    }

f, axs = plt.subplots(1, len(tasks), figsize=(len(tasks)*5 + 1, 3))
i = 0

for task in tasks:
    for model in models:
        ep.plot_task(
            f"f'/content/drive/MyDrive/replicated/probing-results/{size}.json",
            task,
            model,
            model_to_linestyle[model],
            num_layers,
            label=model_to_label[model],
            title=task_to_title[task],
            plot=axs[i]
            )
    i += 1
plt.show()